In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import scipy.io
import matplotlib.pyplot as plt
import os
from scipy import signal
from scipy.fft import fft, fftshift

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

file_names=[]
for dirname, _, filenames in os.walk('/Users/aman/Github bs/datasets/EEG Data/'):
    for filename in filenames:
        file_names.append(os.path.join(dirname, filename))
        #print(os.path.join(dirname, filename))
        
print(file_names)
# each trial is about 54 mins


['/Users/aman/Github bs/datasets/EEG Data/eeg_record7.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record6.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record4.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record5.mat', '/Users/aman/Github bs/datasets/EEG Data/.DS_Store', '/Users/aman/Github bs/datasets/EEG Data/eeg_record1.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record2.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record3.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record16.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record17.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record15.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record29.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record28.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record14.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record10.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record11.mat', '/Users/aman/Github bs/datasets/EEG Data/eeg_record13

In [2]:
# build 5 order high pass filter
from scipy.signal import butter, lfilter, freqz   
# ----- ----- ----- -----    
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    x = signal.filtfilt(b, a, data)
    y = signal.filtfilt(b, a, x)
    return y


In [3]:
marker=128*60*10
#delete file #28 and 14 because it doesnot have enough data
useful_file_index = [3,4,5,6,7,10,11,12,13,17,18,19,20,21,24,25,26,27,31,32,33,34]
#useful_file_index = np.arange(1,35)
useful_channels=[4,5,8,9,10,11,16]
useful_channels_names=['F7','F3','P7','O1','O2','P8','AF4']
chan_num=7
trail_names=[]
data_focus={}
data_unfocus={}
data_drowsy={}
focus={}
unfocus={}
drowsy={}
#for i in useful_file_index:
i=1
for index,filename in enumerate(filenames):
    if int(filename.split('d')[1].split('.')[0]) in useful_file_index:
        mat = scipy.io.loadmat(file_names[index])
        trail_names.append(filename.split('.')[0])
        data_focus[trail_names[-1]]=mat['o']['data'][0,0][0:marker,useful_channels].copy()
        data_unfocus[trail_names[-1]]=mat['o']['data'][0,0][marker:2*marker,useful_channels].copy()
        data_drowsy[trail_names[-1]]=mat['o']['data'][0,0][2*marker:3*marker,useful_channels].copy()
        focus[trail_names[-1]]=mat['o']['data'][0,0][0:marker,useful_channels].copy()
        unfocus[trail_names[-1]]=mat['o']['data'][0,0][marker:2*marker,useful_channels].copy()
        drowsy[trail_names[-1]]=mat['o']['data'][0,0][2*marker:3*marker,useful_channels].copy()

In [4]:
type(data_focus['eeg_record3'])
print(data_focus['eeg_record3'].shape)

(76800, 7)


In [5]:
print(trail_names)

['eeg_record7', 'eeg_record6', 'eeg_record4', 'eeg_record5', 'eeg_record3', 'eeg_record17', 'eeg_record10', 'eeg_record11', 'eeg_record13', 'eeg_record12', 'eeg_record20', 'eeg_record34', 'eeg_record21', 'eeg_record19', 'eeg_record25', 'eeg_record31', 'eeg_record24', 'eeg_record18', 'eeg_record32', 'eeg_record26', 'eeg_record27', 'eeg_record33']


In [6]:
# High Pass 0.16HZ
row, col = data_focus['eeg_record3'].shape
for name in trail_names:
    for i in range(col):
        data_focus[name][:,i]=butter_highpass_filter(data_focus[name][:,i], 0.16, 128, 5)
        data_unfocus[name][:,i]=butter_highpass_filter(data_unfocus[name][:,i], 0.16, 128, 5)
        data_drowsy[name][:,i]=butter_highpass_filter(data_drowsy[name][:,i], 0.16, 128, 5)
        #print(name,data_drowsy[name][:,i].shape)

In [7]:
type(data_focus['eeg_record3'])
print(data_focus['eeg_record3'].shape)

(76800, 7)


In [9]:
feature_names = []
freq_range=np.arange(0.5,18.5,0.5)
symb='_'
#useful_channels_names=['F7','F3','P7','O1','O2','P8','AF4']
for index,channel in enumerate(useful_channels_names):
    for freq in freq_range:
        feature_names.append(channel+symb+str(freq))
feature_names

['F7_0.5',
 'F7_1.0',
 'F7_1.5',
 'F7_2.0',
 'F7_2.5',
 'F7_3.0',
 'F7_3.5',
 'F7_4.0',
 'F7_4.5',
 'F7_5.0',
 'F7_5.5',
 'F7_6.0',
 'F7_6.5',
 'F7_7.0',
 'F7_7.5',
 'F7_8.0',
 'F7_8.5',
 'F7_9.0',
 'F7_9.5',
 'F7_10.0',
 'F7_10.5',
 'F7_11.0',
 'F7_11.5',
 'F7_12.0',
 'F7_12.5',
 'F7_13.0',
 'F7_13.5',
 'F7_14.0',
 'F7_14.5',
 'F7_15.0',
 'F7_15.5',
 'F7_16.0',
 'F7_16.5',
 'F7_17.0',
 'F7_17.5',
 'F7_18.0',
 'F3_0.5',
 'F3_1.0',
 'F3_1.5',
 'F3_2.0',
 'F3_2.5',
 'F3_3.0',
 'F3_3.5',
 'F3_4.0',
 'F3_4.5',
 'F3_5.0',
 'F3_5.5',
 'F3_6.0',
 'F3_6.5',
 'F3_7.0',
 'F3_7.5',
 'F3_8.0',
 'F3_8.5',
 'F3_9.0',
 'F3_9.5',
 'F3_10.0',
 'F3_10.5',
 'F3_11.0',
 'F3_11.5',
 'F3_12.0',
 'F3_12.5',
 'F3_13.0',
 'F3_13.5',
 'F3_14.0',
 'F3_14.5',
 'F3_15.0',
 'F3_15.5',
 'F3_16.0',
 'F3_16.5',
 'F3_17.0',
 'F3_17.5',
 'F3_18.0',
 'P7_0.5',
 'P7_1.0',
 'P7_1.5',
 'P7_2.0',
 'P7_2.5',
 'P7_3.0',
 'P7_3.5',
 'P7_4.0',
 'P7_4.5',
 'P7_5.0',
 'P7_5.5',
 'P7_6.0',
 'P7_6.5',
 'P7_7.0',
 'P7_7.5',
 'P7_8.0'

In [10]:
# STFT was then calculated at a time step of 1 s producing a set of time-varying DFT 
# amplitudes X STFT (t,ω) at 1s intervals within each input EEG channel.
t_win = np.arange(0,128)
M = 128
window_blackman=0.42-0.5*np.cos((2*np.pi*t_win)/(M-1))+0.08*np.cos((4*np.pi*t_win)/(M-1)) #window_blackman = signal.windows.blackmanharris(128)

#col is 7
power_focus = {}
for name in trail_names:
    power_focus[name]=np.zeros([col,513,601])
    
power_unfocus = {}
for name in trail_names:
    power_unfocus[name]=np.zeros([col,513,601])
    
power_drowsy = {}
for name in trail_names:
    power_drowsy[name]=np.zeros([col,513,601])

#the output of the stft is 513*601,1 second data will produce 1 column of data,there are 601
for name in trail_names:
    for i in range(col):
        f, t,y1=scipy.signal.stft(data_focus[name][:,i],fs=128, window=window_blackman, nperseg=128, 
                      noverlap=0, nfft=1024, detrend=False,return_onesided=True, boundary='zeros',
                      padded=True)
        f, t,y2=scipy.signal.stft(data_unfocus[name][:,i],fs=128, window=window_blackman, nperseg=128, 
                      noverlap=0, nfft=1024, detrend=False,return_onesided=True, boundary='zeros',
                      padded=True)
        f, t,y3=scipy.signal.stft(data_drowsy[name][:,i],fs=128, window=window_blackman, nperseg=128, 
                      noverlap=0, nfft=1024, detrend=False,return_onesided=True, boundary='zeros',
                      padded=True)
        power_focus[name][i,:,:]=(np.abs(y1))**2
        power_unfocus[name][i,:,:]=(np.abs(y2))**2
        power_drowsy[name][i,:,:]=(np.abs(y3))**2



In [11]:
type(power_drowsy['eeg_record3'])
print(power_drowsy['eeg_record3'].shape)

(7, 513, 601)


In [12]:
# combine bins into 0.5HZ, and keep 0-18 HZ.

num=[]

power_focus_bin = {}
for name in trail_names:
    power_focus_bin[name]=np.zeros([7,36,601])
    
power_unfocus_bin = {}
for name in trail_names:
    power_unfocus_bin[name]=np.zeros([7,36,601])
    
power_drowsy_bin = {}
for name in trail_names:
    power_drowsy_bin[name]=np.zeros([7,36,601])

for name in trail_names:
    for chn in range(col):
        j=0
        for i in range(1,144,4):
            power_focus_bin[name][chn,j,:]=np.average(power_focus[name][chn,i:i+4,:],axis=0)
            power_unfocus_bin[name][chn,j,:]=np.average(power_unfocus[name][chn,i:i+4,:],axis=0)
            power_drowsy_bin[name][chn,j,:]=np.average(power_drowsy[name][chn,i:i+4,:],axis=0)
            #print(np.average(power_drowsy[name][chn,i:i+4,:],axis=0).shape)
            #if name=='eeg_record3':
            #    if chn==0:
            #        num.append((f[i:i+4]))
            #    print(j)
            j=j+1
            
#print(num)    
#print(len(num))

In [13]:
type(power_drowsy_bin['eeg_record3'])
print(power_drowsy_bin['eeg_record3'].shape)

(7, 36, 601)


In [14]:
# avarage over 15 seconds running window.

power_focus_ave = {}
for name in trail_names:
    power_focus_ave[name]=np.zeros([7,36,585])
    
power_unfocus_ave = {}
for name in trail_names:
    power_unfocus_ave[name]=np.zeros([7,36,585])
    
power_drowsy_ave = {}
for name in trail_names:
    power_drowsy_ave[name]=np.zeros([7,36,585])

for name in trail_names:
    for chn in range(col):
        j=0
        for k in range(0,585):
            power_focus_ave[name][chn,:,j]=np.average(power_focus_bin[name][chn,:,k:k+15],axis=1)
            power_unfocus_ave[name][chn,:,j]=np.average(power_unfocus_bin[name][chn,:,k:k+15],axis=1)
            power_drowsy_ave[name][chn,:,j]=np.average(power_drowsy_bin[name][chn,:,k:k+15],axis=1)
            #print(np.average(power_drowsy_bin[name][chn,:,k:k+15],axis=1).shape)
            j=j+1

In [15]:
type(power_drowsy_ave['eeg_record3'])
print(power_drowsy_ave['eeg_record3'].shape)

(7, 36, 585)


In [16]:
# Turn the data into a vector 
#[252,585]

svm_focus = {}
for name in trail_names:
    svm_focus[name]=np.zeros([252,585])
    
svm_unfocus = {}
for name in trail_names:
    svm_unfocus[name]=np.zeros([252,585])
    
svm_drowsy = {}
for name in trail_names:
    svm_drowsy[name]=np.zeros([252,585])

for name in trail_names:
    for j in range(585):      
        svm_focus[name][:,j]=power_focus_ave[name][:,:,j].reshape(1,-1)
        svm_unfocus[name][:,j]=power_unfocus_ave[name][:,:,j].reshape(1,-1)
        svm_drowsy[name][:,j]=power_drowsy_ave[name][:,:,j].reshape(1,-1)
    svm_focus[name]=10*np.log(svm_focus[name])
    svm_unfocus[name]=10*np.log(svm_unfocus[name])
    svm_drowsy[name]=10*np.log(svm_drowsy[name])
# now, we get the svm vector 252*585 252 rows


In [17]:
type(svm_focus['eeg_record3'])
print(svm_focus['eeg_record3'].shape)

(252, 585)


Using PCA first then SVC, labeling the data into 3 classes.

focused -- 2
unfocused -- 1
drowsy -- 0


In [18]:
#--------0
label_focus = [0]*585
#--------1
label_unfocus = [1]*585
#--------2
label_drowsy = [2]*585

#subject is the variable for all participants

subj1_files={'eeg_record3','eeg_record4','eeg_record5','eeg_record6','eeg_record7'}
subj2_files={'eeg_record10','eeg_record11','eeg_record12','eeg_record13'}
subj3_files={'eeg_record17','eeg_record18','eeg_record19','eeg_record20','eeg_record21'}
subj4_files={'eeg_record24','eeg_record25','eeg_record26','eeg_record27'}
subj5_files={'eeg_record31','eeg_record32','eeg_record33','eeg_record34'}


In [20]:
# I will try to use the data from all participants to train the model
target=[]
subj=np.array([]).reshape(252,0).copy()
for name in trail_names:
    subj=np.concatenate((subj,svm_focus[name]), axis=1)
    subj=np.concatenate((subj,svm_unfocus[name]), axis=1)
    subj=np.concatenate((subj,svm_drowsy[name]), axis=1)  
    target = target+label_focus+label_unfocus+label_drowsy
subj=subj.T
target = np.array(target)

In [50]:
subj.shape

(38610, 252)

In [21]:
print('length of the target:',len(target))
print('the shape of the data from the subject1:', subj.shape)

length of the target: 38610
the shape of the data from the subject1: (38610, 252)


In [22]:
# This part I only train the data for subject1
target1=[]
subj1=np.array([]).reshape(252,0).copy()
for name in subj1_files:
    subj1=np.concatenate((subj1,svm_focus[name]), axis=1)
    subj1=np.concatenate((subj1,svm_unfocus[name]), axis=1)
    subj1=np.concatenate((subj1,svm_drowsy[name]), axis=1)  
    target1 = target1+label_focus+label_unfocus+label_drowsy
subj1=subj1.T
target1 = np.array(target1)

In [53]:
print(subj1.shape)
print(len(target1))

(8775, 252)
8775


In [56]:
print(target1)

[0 0 0 ... 2 2 2]


In [23]:
#for subject 2
target2=[]
subj2=np.array([]).reshape(252,0).copy()
for name in subj2_files:
    subj2=np.concatenate((subj2,svm_focus[name]), axis=1)
    subj2=np.concatenate((subj2,svm_unfocus[name]), axis=1)
    subj2=np.concatenate((subj2,svm_drowsy[name]), axis=1)  
    target2 = target2+label_focus+label_unfocus+label_drowsy
subj2=subj2.T
target2 = np.array(target2)

In [ ]:
print(subj1)

In [26]:
# Train the data from subject1
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC

data_train, data_test, data_train_target, data_test_target = train_test_split(subj1, target1, test_size=0.8, random_state=0)
scaler = preprocessing.StandardScaler().fit(data_train)
X_train_scaled = scaler.transform(data_train)
X_test_scaled = scaler.transform(data_test)

svc = SVC(kernel='rbf')
svc.fit(X_train_scaled,data_train_target)
print(f'The Score for Training data with SVM Model for subject1:',svc.score(X_train_scaled,data_train_target))
print(f'Score of For Test data with SVM Model for subject1 : {svc.score(X_test_scaled,data_test_target)}')

The Score for Training data with SVM Model for subject1: 0.9646723646723647
Score of For Test data with SVM Model for subject1 : 0.9331908831908832


In [ ]:
# Train the data from all subjects
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
training_score=[]
testing_score=[]
split_ratios=[0.6,0.7,0.8]
for i in split_ratios:
    data_train, data_test, data_train_target, data_test_target = train_test_split(subj, target, test_size=i, random_state=0)
    scaler = preprocessing.StandardScaler().fit(data_train)
    X_train_scaled = scaler.transform(data_train)
    X_test_scaled = scaler.transform(data_test)
    split_ratios.append(i)
    svc = SVC(kernel='rbf')
    svc.fit(X_train_scaled,data_train_target)
    training_score.append(svc.score(X_train_scaled,data_train_target))
    testing_score.append(svc.score(X_test_scaled,data_test_target))

In [ ]:
def plot_train_test_accuracy(splits, train_accuracies, test_accuracies):
    plt.figure(figsize=(10, 5))

    plt.plot(splits, train_accuracies, marker='o', label='Training Accuracy')
    plt.plot(splits, test_accuracies, marker='o', label='Testing Accuracy')
    plt.xlabel('Splitting Ratio')
    plt.ylabel('Accuracy')
    plt.title('Training and Testing Accuracy vs Splitting Ratio')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_train_test_accuracy(split_ratios, training_score, testing_score)

In [ ]:
from sklearn.decomposition import PCA

var_explained=[]
split_ratios=[0.6,0.7,0.8,0.9]
for i in split_ratios:
    data_train, data_test, data_train_target, data_test_target = train_test_split(subj, target, test_size=i, random_state=0)
    scaler = preprocessing.StandardScaler().fit(data_train)
    X_train_scaled = scaler.transform(data_train)
    X_test_scaled = scaler.transform(data_test)
    #PCA should be used on scaled data
    pca = PCA()
    pca.fit(X_train_scaled)
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    var = pca.explained_variance_/pca.explained_variance_.sum()
    var_explained.append(var[0:20].sum())

In [ ]:
def plot_train_test_accuracy(splits, var_explained):
    plt.figure(figsize=(10, 5))
    plt.plot(splits, var_explained, marker='o', label='Testing Accuracy')
    plt.xlabel('Splitting Ratio')
    plt.ylabel('Proportion of var explained')
    plt.title('Training and Testing Accuracy vs Splitting Ratio')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_train_test_accuracy(split_ratios, var_explained)

In [ ]:
import seaborn as sns
#sns.heatmap(pca.components_)
#print(sum(pca.components_[:,0]**2))

pca_map=pd.DataFrame(pca.components_,columns=feature_names,index=np.arange(1,253))
#pca.components_.shape
pca_map

In [ ]:
#USE SVM linear model
svc = SVC(kernel='linear')
svc.fit(X_train_scaled,data_train_target)
print(f'The Score for Training data with SVM Linear Model for all subjects:',svc.score(X_train_scaled,data_train_target))
print(f'Score of For Test data with SVM Linear Model for all subjects : {svc.score(X_test_scaled,data_test_target)}')

In [ ]:
#USE SVM linear model
svc = SVC(kernel='rbf')
svc.fit(X_train_scaled,data_train_target)
print(f'The Score for Training data with SVM Linear Model for all subjects:',svc.score(X_train_scaled,data_train_target))
print(f'Score of For Test data with SVM Linear Model for all subjects : {svc.score(X_test_scaled,data_test_target)}')

In [ ]:
#Try KNN Model
from sklearn.neighbors import KNeighborsClassifier
neighbor = KNeighborsClassifier(n_neighbors=3)
neighbor.fit(X_train_scaled,data_train_target)
print("the score for training with data from 5 participants:",neighbor.score(X_train_scaled,data_train_target))
print("the score for test data from 5 participants:",neighbor.score(X_test_scaled,data_test_target))

In [ ]:
# decision trees
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=16)
dt.fit(X_train_scaled,data_train_target)
print(f"the score for training with data from 5 participants:{dt.score(X_train_scaled,data_train_target)}")
print("the score for test data from 5 participants:",dt.score(X_test_scaled,data_test_target))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_scores = []
test_scores = []
for depth in range(1,35):
    dt_reg = RandomForestClassifier(max_depth=depth,random_state=0)
    dt_reg.fit(X_train_scaled,data_train_target)
    train_scores = train_scores +[dt_reg.score(X_train_scaled,data_train_target)]
    test_scores = test_scores + [dt_reg.score(X_test_scaled,data_test_target)]
    
x = list(range(1,35))
plt.plot(x,train_scores,c='r',label='train')
plt.plot(x,test_scores,c='b',label='test')
plt.xlabel('depth')
plt.ylabel('accuracy')
plt.title('Depth vs. Accuracy for Random Forest Classifier')
plt.show()

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train_pca[:,0:30],data_train_target)
print(f'The Score for Training data with SVM RBF Model for all subjects:',svc.score(X_train_pca[:,0:30],data_train_target))
print(f'Score of For Test data with SVM RBF Model for all subjects : {svc.score(X_test_pca[:,0:30],data_test_target)}')

In [ ]:
svc = SVC(kernel='linear')
svc.fit(X_train_pca[:,0:30],data_train_target)
print(f'The Score for Training data with SVM RBF Model for all subjects:',svc.score(X_train_pca[:,0:30],data_train_target))
print(f'Score of For Test data with SVM RBF Model for all subjects : {svc.score(X_test_pca[:,0:30],data_test_target)}')

In [ ]:
#KNN after PCA
neighbor.fit(X_train_pca[:,0:30],data_train_target)
print("the score for training with data from 5 participants:",neighbor.score(X_train_pca[:,0:30],data_train_target))
print("the score for test data from 5 participants:",neighbor.score(X_test_pca[:,0:30],data_test_target))

In [ ]:
# 5 fold validation
from sklearn.model_selection import KFold
import tensorflow as tf
tf.random.set_seed(0)
kfold = KFold(n_splits=5, shuffle=True)

data_train_target=data_train_target

fold_no = 1

score_tr=[]
score_cv=[]

svm = SVC(kernel='rbf')

for train,cv in kfold.split(X_train_scaled,data_train_target):
    X_tr = X_train_scaled[train]
    Y_tr = data_train_target[train]
    X_cv = X_train_scaled[cv]
    Y_cv = data_train_target[cv]
    
    tf.random.set_seed(0)
    
    svm.fit(X_tr,Y_tr)
    score_tr.append(svm.score(X_tr,Y_tr))
    score_cv.append(svm.score(X_cv,Y_cv))
    
    print(f'Score for {fold_no} Fold Training: {score_tr[-1]:.3f}')
    print(f'Score for {fold_no} Fold cv    : {score_cv[-1]:.3f}')
    print('----------------------------------')
    fold_no = fold_no + 1
    
print(f'Score of Average For Training: {np.mean(score_tr):.3f}')
print(f'Score of Average For CV.     : {np.mean(score_cv):.3f}')